In [1]:
# Для выполнения этого задания вам понадобятся данные о кредитных историях клиентов одного из банков.
# Поля в предоставляемых данных имеют следующий смысл:
#    LIMIT_BAL: размер кредитного лимита (в том числе и на семью клиента)
#    SEX: пол клиента (1 = мужской, 2 = женский )
#    EDUCATION: образование (0 = доктор, 1 = магистр; 2 = бакалавр; 3 = выпускник школы; 4 = начальное образование; 5= прочее; 6 = нет данных ).
#    MARRIAGE: (0 = отказываюсь отвечать; 1 = замужем/женат; 2 = холост; 3 = нет данных).
#    AGE: возраст в годах
#    PAY_0 - PAY_6 : История прошлых платежей по кредиту. PAY_6 - платеж в апреле, ... Pay_0 - платеж в сентябре.
#       Платеж = (0 = исправный платеж, 1=задержка в один месяц, 2=задержка в 2 месяца ...)
#    BILL_AMT1 - BILL_AMT6: задолженность, BILL_AMT6 - на апрель, BILL_AMT1 - на сентябрь
#    PAY_AMT1 - PAY_AMT6: сумма уплаченная в PAY_AMT6 - апреле, ..., PAY_AMT1 - сентябре
#    default - индикатор невозврата денежных средств

In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

In [3]:
frame = pd.read_csv("/content/drive/MyDrive/dataset/credit_card_default_analysis.csv", sep=",", header=0)

In [4]:
frame.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000,2,2,1,24,2,2,0,0,0,0,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,0,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,0,0,0,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


Гипотеза о равенстве медианных значений кредитного лимита с помощью подходящей интервальной оценки

In [5]:
def stat_intervals(stat, alpha=0.05):
    """Доверительный интервал"""
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

def get_bootstrap_samples(data, n_samples):
    """Бутстрап-выборка"""
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [6]:
#Разделим данные на две группы default и success
default_group = frame[frame["default"] == 1]
success_group = frame[frame["default"] == 0]
default_group_limits = default_group["LIMIT_BAL"].values
success_group_limits = success_group["LIMIT_BAL"].values

In [7]:
#Докажем, с помощью доверительных интервалов, построенных с помощью метода bootstrap, что различие между значениями в этих группах действительно есть.

In [8]:
np.random.seed(0)

group1_median = list(map(np.median, get_bootstrap_samples(default_group_limits, 1000)))
group2_median  = list(map(np.median, get_bootstrap_samples(success_group_limits, 1000)))

In [9]:
delta_median = list(map(lambda x: x[1] - x[0], zip(group1_median, group2_median)))

In [10]:
print ("Макc значение для для группы1:", np.max(default_group_limits),"Мин значение для для группы1:", np.min(default_group_limits))
print ("Макc значение для для группы2:", np.max(success_group_limits),"Мин значение для для группы2:", np.min(success_group_limits ))
print ("95% доверительный интервал для группы1:", stat_intervals(group1_median, 0.05))
print ("95% доверительный интервал для группы2:",  stat_intervals(group2_median, 0.05))
print ("95% доверительный интервал для разности медиан:",  stat_intervals(delta_median, 0.05))

Макc значение для для группы1: 740000 Мин значение для для группы1: 10000
Макc значение для для группы2: 1000000 Мин значение для для группы2: 10000
95% доверительный интервал для группы1: [80000. 90000.]
95% доверительный интервал для группы2: [150000. 150000.]
95% доверительный интервал для разности медиан: [60000. 70000.]


Гипотеза о том, что гендерный состав группы людей вернувших и не вернувших кредит отличается.

In [11]:
default_group_sex = default_group["SEX"].values
success_group_sex = success_group["SEX"].values

In [12]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(len(sample1[sample1==1])) / n1
    p2 = float(len( sample2[sample2==1])) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [13]:
proportions_diff_z_stat_ind(default_group_sex, success_group_sex)

6.921375088562897

In [14]:
def proportions_diff_z_test(sample1, sample2, alternative = 'two-sided'):
    z_stat = proportions_diff_z_stat_ind(sample1, sample2)
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [15]:
pvalue = proportions_diff_z_test(default_group_sex, success_group_sex, alternative='two-sided')
print ("p-value: ", pvalue)

p-value:  4.472866521609831e-12


In [16]:
#Достигаемый уровень значимости Z-критерия p-value для этих двух выборок сильно меньше 0.05. 
#Гипотеза о равенстве доли мужчин в выборках "default" и "success" с уверенностью отвергается.